In [1]:
import random
from random import randint

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense,LSTM, Concatenate, Input, Conv1D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling1D, MaxPooling1D, GlobalAveragePooling1D, UpSampling1D, concatenate, Conv1DTranspose, Cropping1D, UpSampling1D, ZeroPadding1D, Reshape
from keras.models import Sequential,Model
from matplotlib.pyplot import legend
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.linalg.linear_operator_algebra import inverse

from generator import Generator
from keras import metrics
import importlib

from inverse import fit_linear

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from datasetLoader import *

In [3]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()

In [4]:
with open("data/initial kernels/Kernel_Silica_Adsorption.npy", 'rb') as f:
    data_sorb = np.load(f)
    data_sorb_tensor = tf.constant((data_sorb.T[87:]).T)
    data_sorb_tensor = tf.cast(data_sorb_tensor, tf.float32)

def custom_loss(y_true, y_pred):
    pass
    #print("data_sorb_tensor.shape = ", data_sorb_tensor.shape, y_pred[:,:128].shape)
    isotherm_from_distribution = tf.tensordot(y_pred[:,:128], data_sorb_tensor, axes=1)
    #print("isotherm_from_distribution = ", isotherm_from_distribution.shape, y_pred[:,128:].shape)
    return tf.reduce_sum(tf.square(y_pred[:,:128] - y_true[:,:128])) + 0*tf.reduce_sum(tf.square(y_pred[:,128:] - isotherm_from_distribution))


# np.ediff1d(pore_widths, to_begin=pore_widths[0])

In [5]:
import tensorflow as tf

class ConstrainedOutputLayer(tf.keras.layers.Layer):
    def __init__(self, M=None, weight=10, **kwargs):
        super(ConstrainedOutputLayer, self).__init__(**kwargs)
        # Initialize M as None to handle custom initialization in from_config
        if M is not None:
            self.M = tf.constant(M, dtype=tf.float32)
        else:
            self.M = None  # Will be set later during call or from_config
        self.weight = weight

    def call(self, y_out, x_in):
        if self.M is None:
            raise ValueError("Matrix M must be initialized.")
        # Apply the constraint logic
        constraint = tf.matmul(y_out, self.M) - x_in
        self.add_loss(self.weight * tf.reduce_mean(tf.square(constraint)))
        return y_out

    def get_config(self):
        # Get the base config and update it with custom arguments
        config = super(ConstrainedOutputLayer, self).get_config()
        # Serialize M as a list, and weight as is
        config.update({
            'M': self.M.numpy().tolist() if self.M is not None else None,
            'weight': self.weight
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Reconstruct the layer from the config
        M = tf.constant(config['M'], dtype=tf.float32) if config['M'] is not None else None
        weight = config['weight']
        return cls(M=M, weight=weight)


In [6]:
def create_model(x):
    layer = []
    layer.append(Input(shape=x[0].shape))
    layer.append(Dense(200, activation='relu')(layer[-1]))
    layer.append(Dense(200, activation='relu')(layer[-1]))
    layer.append(Dense(200, activation='relu')(layer[-1]))
    layer.append(Dense(200, activation='relu')(layer[-1]))
    layer.append(Dense(128, activation='relu')(layer[-1]))
    layer.append(Dense(128, activation='relu')(layer[-1]))
    layer.append(Dense(128, activation='relu')(layer[-1]))
    layer.append(Dense(128, activation='relu')(layer[-1]))
    layer.append(Concatenate()([layer[-1], layer[0]]))
    model = tf.keras.Model(inputs=layer[0], outputs=layer[-1])
    model.compile(loss=custom_loss, optimizer='Adam')
    #model.compile(loss='mean_squared_error', optimizer='Adam')
    return model

def create_model_CNN_output():
    input_length = 371    # Длина входной последовательности
    output_length = 128   # Длина выходной последовательности
    feature_size = 1     # Количество признаков на каждом шаге (1 для одномерной последовательности)

    # === ЭНКОДЕР ===
    inputs = Input(shape=(input_length, feature_size))  # Входная последовательность
    x = Conv1D(16, kernel_size=3, activation='relu', padding='same')(inputs)  # Первый свёрточный слой
    x = Conv1D(32, kernel_size=3, activation='relu', padding='same')(x)  # Второй свёрточный слой
    x = GlobalAveragePooling1D()(x)  # Сжатие признаков (получаем скрытое представление)

    # === ПРОЕКЦИОННЫЙ СЛОЙ ===
    x = Dense(32, activation='relu')(x)  # Преобразование скрытого представления
    x = Dense(output_length * 64, activation='relu')(x)  # Подготовка к декодеру
    x = Reshape((output_length, 64))(x)  # Формирование "последовательности" для декодера

    # === ДЕКОДЕР ===
    x = Conv1DTranspose(64, kernel_size=3, activation='relu', padding='same')(x)  # Первый слой декодера
    x = Conv1DTranspose(32, kernel_size=3, activation='relu', padding='same')(x)  # Второй слой декодера
    outputs = Conv1D(1, kernel_size=1, activation='relu', padding='same')(x)  # Финальный слой

    # === МОДЕЛЬ ===
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

def create_model_CNN():
    inputs = Input(shape=(448, 1), name="Input")
    x = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same', name="c1")(inputs)
    x = MaxPooling1D(pool_size=2, padding='same', name="p1")(x)
    x = BatchNormalization()(x)


    x = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same', name="c2")(x)
    x = MaxPooling1D(pool_size=2, padding='same', name="p2")(x)
    x = BatchNormalization()(x)
    
    x = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', name="c3")(x)
    x = MaxPooling1D(pool_size=2, padding='same', name="p3")(x)
    x = BatchNormalization()(x)
    
    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="c4")(x)
    x = MaxPooling1D(pool_size=2, padding='same', name="p4")(x)
    x = BatchNormalization()(x)
    
    x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', name="c5")(x)
    x = MaxPooling1D(pool_size=2, padding='same', name="p5")(x)
    x = BatchNormalization()(x)
    
    x = Flatten(name="f1")(x)

    #DENSE OUTPUT
    x = Dense(128, activation='relu', name="d1")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.15)(x)
    x = Dense(128, activation='relu', name="d2")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.15)(x)
    x = Dense(128, activation='relu', name="d3")(x)
    #x = ConstrainedOutputLayer(M=data_sorb_tensor)(x, tf.squeeze(inputs, axis=-1))
    
    # input_for_loss = Reshape((371, ))(inputs)
    # outputs = Concatenate()([x, input_for_loss])
    
    #outputs = d3
    
    model = Model(inputs=[inputs], outputs=[x], name="CNN")
    #model.compile(loss='mean_squared_error', optimizer='Adam')
    model.compile(loss="mse", optimizer='Adam')
    return model

In [7]:
# with open("data/datasets/reports_best_tyhanov.npz", 'rb') as f:
#     dataset = np.load(f)
#     isotherm_data = dataset["isotherm_data"]
#     pore_distribution_data = dataset["pore_distribution_data"]
#     pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")

In [8]:
x_experimental, y_experimental = load_dataset('data/datasets/SMP_CUT_NOT_ZERO.npz')

x_experimental_QPS, y_experimental_QPS = load_dataset('data/datasets/reports.npz')

x_generated, y_generated = load_dataset('data/datasets/silica_random.npz')

In [9]:
x_train_exp, x_test_exp, y_train_exp, y_test_exp = train_test_split(x_experimental, y_experimental, test_size=0.15, random_state=1)
x_train_gen, x_test_gen, y_train_gen, y_test_gen = train_test_split(x_generated, y_generated, test_size=0.15, random_state=1)

In [146]:
with open(f'data/datasets/SMP_test.npz', "wb") as f:
    np.savez_compressed(f, x_train=x_train_exp, y_train=y_train_exp, x_test=x_test_exp, y_test=y_test_exp)

In [10]:
# x_train = np.concatenate((x_train_gen, x_train_exp), axis=0)
# x_test = np.concatenate((x_test_gen, x_test_exp), axis=0)
# y_train = np.concatenate((y_train_gen, y_train_exp), axis=0)
# y_test = np.concatenate((y_test_gen, y_test_exp), axis=0)

x_train, x_test, y_train, y_test = train_test_split(x_experimental, y_experimental, test_size=0.15, random_state=1)

In [11]:
model = create_model_CNN()
#model = create_model_CNN_output()
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 448, 1)]          0         
                                                                 
 c1 (Conv1D)                 (None, 448, 8)            32        
                                                                 
 p1 (MaxPooling1D)           (None, 224, 8)            0         
                                                                 
 batch_normalization (BatchN  (None, 224, 8)           32        
 ormalization)                                                   
                                                                 
 c2 (Conv1D)                 (None, 224, 16)           400       
                                                                 
 p2 (MaxPooling1D)           (None, 112, 16)           0         
                                                               

In [147]:
for layer in model.layers:
    layer.trainable = False
for i in range(11, 15):
    model.layers[i].trainable = True
for layer in model.layers:
    print(layer, layer.trainable)
model.summary()

<keras.engine.input_layer.InputLayer object at 0x000001F03BCCD4B0> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001F03B829EA0> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D object at 0x000001EEF68C62F0> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001F03DE0AFE0> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001F03DE0B910> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D object at 0x000001F03DE0B220> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001F03DE0BD00> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001F03DE0AF20> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D object at 0x000001F03DE0AE30> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001F03DE48A30> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001F03DE48AC0> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D ob

In [158]:
# from keras import backend as K
# K.set_value(model.optimizer.learning_rate, 0.001)
model_name = "silica_CNN_cut_psd_exp.keras"
mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath=f'data/models/{model_name}', save_best_only=True,
                                              monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                              save_freq='epoch')

reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                      patience=100, verbose=1, mode='auto')
history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=1500, batch_size=1024, shuffle=True, verbose=2,
                    validation_data=(np.array(x_test), np.array(y_test)), callbacks=[mcp_save])
current_epoch = history.epoch[-1]
plot_loss(history)

Epoch 1/1500

Epoch 1: val_loss improved from inf to 1.20195, saving model to data/models\silica_CNN_cut_psd_exp.keras
6/6 - 0s - loss: 0.0701 - val_loss: 1.2019 - 481ms/epoch - 80ms/step
Epoch 2/1500

Epoch 2: val_loss improved from 1.20195 to 1.07906, saving model to data/models\silica_CNN_cut_psd_exp.keras
6/6 - 0s - loss: 0.0717 - val_loss: 1.0791 - 180ms/epoch - 30ms/step
Epoch 3/1500

Epoch 3: val_loss did not improve from 1.07906
6/6 - 0s - loss: 0.0756 - val_loss: 1.4257 - 156ms/epoch - 26ms/step
Epoch 4/1500

Epoch 4: val_loss did not improve from 1.07906
6/6 - 0s - loss: 0.0686 - val_loss: 1.3540 - 141ms/epoch - 23ms/step
Epoch 5/1500

Epoch 5: val_loss did not improve from 1.07906
6/6 - 0s - loss: 0.0638 - val_loss: 1.3911 - 141ms/epoch - 23ms/step
Epoch 6/1500

Epoch 6: val_loss did not improve from 1.07906
6/6 - 0s - loss: 0.0653 - val_loss: 1.1962 - 156ms/epoch - 26ms/step
Epoch 7/1500

Epoch 7: val_loss did not improve from 1.07906
6/6 - 0s - loss: 0.0611 - val_loss: 1.3

In [87]:
plot_loss(history)

NameError: name 'history' is not defined

In [115]:
# load model
model_name = "silica_CNN_cut_psd_exp.keras"
model = tf.keras.models.load_model(f'data/models/{model_name}', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss, 'ConstrainedOutputLayer': ConstrainedOutputLayer})

In [13]:
#tf.keras.models.save_model(model,'data/models/silica_CNN3.keras')

In [48]:
# prediction = model.evaluate(np.array(x_test), np.array(y_test))
prediction = model.predict(np.array(x_test_exp))
model.evaluate(np.array(x_test), np.array(y_test), verbose=2)

107/107 [==============================] - 0s 2ms/step
107/107 - 0s - loss: 25.3987 - 168ms/epoch - 2ms/step


25.39873695373535

In [163]:
# test on x_train
def fetch_prediction(prediction):
    return prediction[:128]

pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
pressures = np.load("data/initial kernels/Pressure_Silica.npy")
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
for i in range(NX):
    for j in range(NY):
        k = np.random.randint(0, len(x_test_exp)) 
        # axis[i, j].plot(pore_widths , fetch_prediction(prediction[k]), marker=".", label=f"Prediction")
        # axis[i, j].plot(pore_widths , y_test[k], marker=".", label="Real distribution")

        axis[i, j].grid()
        axis[i, j].set_xlabel("nm",fontsize=8)
        axis[i, j].plot(pore_widths, fetch_prediction(prediction[k]), marker=".", label=f"Model PSD")
        axis[i, j].plot(pore_widths, y_test_exp[k], marker=".", label="PSD")
        
        iso_axis = axis[i, j].twiny()
        iso_axis.set_xlabel("P/P$^0$",fontsize=8)
        iso_axis.plot(pressures[:-10], x_test_exp[k], label="Isotherm", color = 'green')
        kernel = (data_sorb.T[:-10])
        iso_axis.plot(pressures[:-10], np.dot(kernel, prediction[k][:128]), label="Isotherm by PSD", color="red")
        axis[i, j].set_title(f"№ {k}")
        axis[i, j].title.set_size(10)
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.9)
plt.legend()
axis[0, 0].legend()
plt.show()

In [ ]:
math_psds = [fit_linear(x_test_exp[i], kernel.T, 0).x for i in range(len(x_test_exp))]
restored_isotherms = [np.dot(kernel, psd) for psd in math_psds]

In [162]:
from tools import model_tester
#importlib.reload(model_tester)
    
model_tester.plot_testing_graphs(prediction, x_test_exp, restored_isotherms, kernel, model_name)

In [124]:
error_lst_math, roughness_lst_math = model_tester.calculate_error_for_math(x_test_exp, restored_isotherms=restored_isotherms)
kde_x_math, kde_error_math, _  = model_tester.calculate_kde_data(error_lst_math, stop=150)
print(model_name)
print("median error:", np.median(error_lst_math))

silica_CNN_cut_psd_exp.keras
average error: 11.554572714305223


In [130]:
plt.plot(kde_x, kde_roughness, label=model_name)
#plt.plot(kde_x_math, kde_error_math, label="math")
plot()

In [ ]:
plt.plot(kde_x, kde_error, label=model_name)
plt.plot(kde_x_math, kde_error_math, label="math")
plot()

In [16]:
# test with test Generator
from tools import TestApp

gen = Generator(path_s="data/initial kernels/Kernel_Silica_Adsorption.npy",
                path_d="data/initial kernels/Kernel_Silica_Desorption.npy",
                path_p_d="data/initial kernels/Pressure_Silica.npy",
                path_p_s="data/initial kernels/Pressure_Silica.npy",
                path_a="data/initial kernels/Size_Kernel_Silica_Adsorption.npy"
                )
# gen = Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
#                               path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
#                               path_p_d="data/initial kernels/Pressure_Carbon.npy",
#                               path_p_s="data/initial kernels/Pressure_Carbon.npy",
#                               path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
#                             )

#TestApp.App(model, gen)

In [17]:
exp_file_list = ["MCM-41", "SBA-15", "SBA-16", "MIL-101", "MIL-101_2", "DUT-49", "FDM-4", "PCN-333", "PCN-777",
                 "MIL-100"]

p_exp_list = []
n_s_exp_raw_list = []
for exp_file_name in exp_file_list:
    data = pd.read_csv(f"data/real/{exp_file_name}.txt", header=None)
    # p_exp_list.append(data.iloc[:,1].to_numpy())
    # n_s_exp_raw_list.append(data.iloc[:,3].to_numpy())
    p_exp_list.append(data.iloc[:, 1].to_numpy())
    n_s_exp_raw_list.append(data.iloc[:, 3].to_numpy())

In [18]:
j = 2
plt.plot(p_exp_list[j], n_s_exp_raw_list[j], marker=".", label=exp_file_list[j])
plot()

In [19]:
# интерполируем экспериментальную изотерму под давления кернала
n_s_exp_list = []
for i in range(len(p_exp_list)):
    n_s_exp_list.append(np.interp(gen.pressures_s[77:367], p_exp_list[i], n_s_exp_raw_list[i]))

In [21]:
j = 2
plt.plot(gen.pressures_s[77:367], n_s_exp_list[j], marker=".", label=exp_file_list[j])
plot()

In [22]:
n_s_exp_for_net_list = [pre_process_isotherm(np.copy(n_s_exp), scale=False) for n_s_exp in n_s_exp_list]
fit_exp_list = [model.predict(np.array([n_s_exp_for_net])).T for n_s_exp_for_net in n_s_exp_for_net_list]
fit_exp_list = [fetch_prediction(i) for i in fit_exp_list]

InvalidArgumentError: Graph execution error:

Detected at node 'CNN/f1/Reshape' defined at (most recent call last):
    File "C:\Users\user\anaconda3\envs\py310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\user\anaconda3\envs\py310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\user\anaconda3\envs\py310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\user\anaconda3\envs\py310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\user\anaconda3\envs\py310\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\user\anaconda3\envs\py310\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\user\anaconda3\envs\py310\lib\asyncio\base

In [23]:
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        x_scale_factor = max(gen.a_array) / max(p_exp_list[k])
        y_scale_factor = max(fit_exp_list[k]) / max(n_s_exp_raw_list[k])
        axis[i, j].plot(pore_widths, fit_exp_list[k], marker=".", label=f"Distribution")
        axis[i, j].plot(p_exp_list[k] * x_scale_factor, n_s_exp_raw_list[k],
                        label=f"{exp_file_list[k]}", marker=".")
        
        kernal = (data_sorb.T[40:])
        axis[i, j].plot(gen.pressures_s[40:458]* x_scale_factor, np.dot(kernal, fit_exp_list[k]), label="Isotherm by distribution")
        
        axis[i, j].set_title(f"max at {round(gen.a_array[np.argmax(fit_exp_list[k])], 2)} nm")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

NameError: name 'fit_exp_list' is not defined

In [ ]:
### Classic
import inverse

kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")

### normalize on size
# a_array = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
# for i in range(len(a_array)):
#     kernel[i] /= a_array[i]
#     kernel[i] /= a_array[i]
###

cut_kernel = []
for i in range(len(kernel)):
    cut_kernel.append(kernel[i][40:458])
cut_kernel = np.array(cut_kernel)
fit_classic_list = [inverse.fit_SLSQP(adsorption=n_s, kernel=cut_kernel, a_array=pore_widths) for n_s in n_s_exp_list]

In [ ]:
from matplotlib import animation

importlib.reload(inverse)


def create_regularization_animation(file):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5, True)
    fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=0)
    line1, = ax.plot(pore_widths[:-30], fit_classic.x[:-30], marker=".", label=f"a = {0}")

    y_scale_factor = max(fit_classic.x) / max(fit_exp_list[2])
    #plt.plot(pore_widths, fit_exp_list[2] * y_scale_factor, marker=".", label=f"Суррогатная модель")

    ax.set_ylabel("Объем пор, $см^3$/ нм * г")
    ax.set_xlabel("Размер пор, нм")

    L = plt.legend(loc=1)  # Define legend objects

    def update(frame):
        a = frame / 4 + 2
        fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=a)
        line1.set_ydata(fit_classic.x[:-30])
        L.get_texts()[0].set_text(
            f"Распределение пор по размерам, параметр регуляризации α = {round(a-2,1)}")  # Update label each at frame
        return line1,

    ani = animation.FuncAnimation(fig=fig, func=update, frames=100, interval=100)
    writervideo = animation.FFMpegWriter(fps=30)
    ani.save(file, writer=writervideo, dpi=200)
    plt.grid()
    plt.show()
create_regularization_animation("SBA-16_regularization.mp4")
def plot_regularization_graphs():
    for a in [1, 5, 10, 50]:
        fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=a)
        plt.plot(pore_widths, fit_classic.x, marker=".", label=f"α = {a}")
    plt.ylabel("Объем пор, $см^3$/ нм * гр")
    plt.xlabel("Размер пор, нм")
    plot()
#plot_regularization_graphs()

In [ ]:
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        x_scale_factor = max(gen.a_array) / max(p_exp_list[k])
        y_scale_factor = max(fit_classic_list[k].x) / max(n_s_exp_raw_list[k])
        y_scale_factor_net = max(fit_classic_list[k].x) / max(fit_exp_list[k])

        axis[i, j].plot(pore_widths, fit_exp_list[k] * y_scale_factor_net, marker=".", label=f"net")
        axis[i, j].plot(pore_widths, fit_classic_list[k].x, marker=".", label=f"classic")
        axis[i, j].plot(p_exp_list[k] * x_scale_factor, n_s_exp_raw_list[k] * y_scale_factor,
                        label=f"{exp_file_list[k]}", marker=".")
        axis[i, j].set_title(f"max at {round(gen.a_array[np.argmax(fit_classic_list[k].x)], 2)} nm")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
### Classic
import inverse

kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")

### normalize on size
# a_array = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
# for i in range(len(a_array)):
#     kernel[i] /= a_array[i]
#     kernel[i] /= a_array[i]
###

cut_kernel = []
for i in range(len(kernel)):
    cut_kernel.append(kernel[i][40:458])
cut_kernel = np.array(cut_kernel)
fit_classic_list = [inverse.fit_SLSQP(adsorption=n_s, kernel=cut_kernel, a_array=pore_widths) for n_s in n_s_exp_list]

In [ ]:
from matplotlib import animation

importlib.reload(inverse)


def create_regularization_animation(file):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5, True)
    fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=0)
    line1, = ax.plot(pore_widths[:-30], fit_classic.x[:-30], marker=".", label=f"a = {0}")

    y_scale_factor = max(fit_classic.x) / max(fit_exp_list[2])
    #plt.plot(pore_widths, fit_exp_list[2] * y_scale_factor, marker=".", label=f"Суррогатная модель")

    ax.set_ylabel("Объем пор, $см^3$/ нм * г")
    ax.set_xlabel("Размер пор, нм")

    L = plt.legend(loc=1)  # Define legend objects

    def update(frame):
        a = frame / 4 + 2
        fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=a)
        line1.set_ydata(fit_classic.x[:-30])
        L.get_texts()[0].set_text(
            f"Распределение пор по размерам, параметр регуляризации α = {round(a-2,1)}")  # Update label each at frame
        return line1,

    ani = animation.FuncAnimation(fig=fig, func=update, frames=100, interval=100)
    writervideo = animation.FFMpegWriter(fps=30)
    ani.save(file, writer=writervideo, dpi=200)
    plt.grid()
    plt.show()
create_regularization_animation("SBA-16_regularization.mp4")
def plot_regularization_graphs():
    for a in [1, 5, 10, 50]:
        fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=a)
        plt.plot(pore_widths, fit_classic.x, marker=".", label=f"α = {a}")
    plt.ylabel("Объем пор, $см^3$/ нм * гр")
    plt.xlabel("Размер пор, нм")
    plot()
#plot_regularization_graphs()

In [ ]:
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        x_scale_factor = max(gen.a_array) / max(p_exp_list[k])
        y_scale_factor = max(fit_classic_list[k].x) / max(n_s_exp_raw_list[k])
        y_scale_factor_net = max(fit_classic_list[k].x) / max(fit_exp_list[k])

        axis[i, j].plot(pore_widths, fit_exp_list[k] * y_scale_factor_net, marker=".", label=f"net")
        axis[i, j].plot(pore_widths, fit_classic_list[k].x, marker=".", label=f"classic")
        axis[i, j].plot(p_exp_list[k] * x_scale_factor, n_s_exp_raw_list[k] * y_scale_factor,
                        label=f"{exp_file_list[k]}", marker=".")
        axis[i, j].set_title(f"max at {round(gen.a_array[np.argmax(fit_classic_list[k].x)], 2)} nm")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
# compare net, classic, quantachrome distributions
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
k = 0


def calculate_isotherm_by_distribution(generator: Generator, a_array, distribution):
    generator.a_array = a_array
    generator.pore_distribution = distribution
    generator.calculate_isotherms()
    return generator.n_s


#/np.ediff1d(pore_widths, to_begin=pore_widths[0])
for i in range(NX):
    for j in range(NY):
        quantachrome_pore_size = \
            np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[
                0] / 10 * 2  # /10 - перевод в НМ * 2 - в QH размер - Half pore width.
        quantachrome_dV = \
            np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[3]
        # isotherm_formQC = calculate_isotherm_by_distribution(gen, pore_widths, np.interp(pore_widths, quantachrome_pore_size, quantachrome_dV))
        # y_scale_factor_QH = max(n_s_exp_raw_list[k]) / max(isotherm_formQC)
        # quantachrome_dV *= y_scale_factor_QH

        y_scale_factor_classic = max(quantachrome_dV) / max(fit_classic_list[k].x)
        y_scale_factor_net = max(quantachrome_dV) / max(fit_exp_list[k])

        axis[i, j].plot(pore_widths, fit_exp_list[k] * y_scale_factor_net, marker=".", label=f"net")
        axis[i, j].plot(pore_widths, fit_classic_list[k].x * y_scale_factor_classic, marker=".", label=f"classic")
        axis[i, j].plot(quantachrome_pore_size, quantachrome_dV, marker=".", label=f"quantachrome")
        axis[i, j].set_title(f"{exp_file_list[k]}")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
# plots for presentation
import matplotlib.pyplot as plt
from importlib import reload

plt = reload(plt)

k = 0

quantachrome_pore_size = \
    np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[
        0] / 10 * 2  # /10 - перевод в НМ * 2 - в QH размер - Half pore width.
quantachrome_dV = \
    np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[3]
# isotherm_formQC = calculate_isotherm_by_distribution(gen, pore_widths, np.interp(pore_widths, quantachrome_pore_size, quantachrome_dV))
# y_scale_factor_QH = max(n_s_exp_raw_list[k]) / max(isotherm_formQC)
# quantachrome_dV *= y_scale_factor_QH

y_scale_factor_classic = max(quantachrome_dV) / max(fit_classic_list[k].x)
y_scale_factor_net = max(quantachrome_dV) / max(fit_exp_list[k])

plt.plot(pore_widths, fit_exp_list[k] * y_scale_factor_net, marker=".", label=f"Суррогатная модель")
plt.plot(pore_widths, fit_classic_list[k].x * y_scale_factor_classic, marker=".", label=f"Математическое решение")
#plt.plot(quantachrome_pore_size, quantachrome_dV, marker=".", label=f"Математическое решение QH") 
plt.legend(loc="upper right")
plt.grid()
plt.title(f"{exp_file_list[k]}")

# plt.xscale('log')
plt.subplots_adjust(left=0.15,
                    bottom=0.133,
                    right=0.979,
                    top=0.917,
                    wspace=0.4,
                    hspace=0.4)
plt.legend()
plt.grid(True)
plt.ylabel("Объем пор, $см^3$/ нм * гр")
plt.xlabel("Размер пор, нм")
plt.show()


In [ ]:
# plots for presentation 2
k = 3
plt.plot(p_exp_list[k], n_s_exp_raw_list[k], marker=".", color='b', label=f"{exp_file_list[k]}")
k = 9
plt.plot(p_exp_list[k], n_s_exp_raw_list[k], marker=".", color='r', label=f"{exp_file_list[k]}")
plt.legend(loc="upper right")
plt.grid()
plt.title(f"Изотермы адсорбции")

# plt.xscale('log')
plt.subplots_adjust(left=0.15,
                    bottom=0.133,
                    right=0.979,
                    top=0.917,
                    wspace=0.4,
                    hspace=0.4)
plt.legend()
plt.grid(True)
plt.ylabel("Адсорбция, $см^3$/г")
plt.xlabel("Давление, $P/P_{0}$")
plt.show()


In [ ]:
# plots for presentation 3

fig, ax1 = plt.subplots(figsize=(6, 7))
ax1.set_xlabel("Размер пор, нм")
ax1.set_ylabel("Объем пор, $см^3$/ нм * гр")
k = 0
plt.title(f"{exp_file_list[k]}")
quantachrome_pore_size = \
    np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[
        0] / 10 * 2  # /10 - перевод в НМ * 2 - в QH размер - Half pore width.
quantachrome_dV = \
    np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[3]
# isotherm_formQC = calculate_isotherm_by_distribution(gen, pore_widths, np.interp(pore_widths, quantachrome_pore_size, quantachrome_dV))
# y_scale_factor_QH = max(n_s_exp_raw_list[k]) / max(isotherm_formQC)
# quantachrome_dV *= y_scale_factor_QH

y_scale_factor_classic = max(quantachrome_dV) / max(fit_classic_list[k].x)
y_scale_factor_net = max(quantachrome_dV) / max(fit_exp_list[k])

ax1.plot(pore_widths[0:100], (fit_exp_list[k] * y_scale_factor_net)[0:100], marker=".", label=f"Суррогатная модель")
ax1.plot(pore_widths[0:100], (fit_classic_list[k].x * y_scale_factor_classic)[0:100], marker=".",
         label=f"Математическое решение")
# ax1.tick_params(axis='y')
# plt.legend(loc="right")


ax2 = ax1.twinx()
ax3 = ax2.twiny()  # instantiate a second axes that shares the same x-axis
ax3.set_xlabel("Давление, $P/P_{0}$")
ax2.set_ylabel("Адсорбция, $см^3$/г")  # we already handled the x-label with ax1
ax3.plot(p_exp_list[k], n_s_exp_raw_list[k], color='g', label=f"Изотерма адсорбции")
# plt.legend(loc="right")
# 
# ax2.tick_params(axis='y')
# plt.subplots_adjust(left=0.15,
#                     bottom=0.133, 
#                     right=0.979, 
#                     top=0.917, 
#                     wspace=0.4, 
#                     hspace=0.4)

fig.tight_layout()
plt.show()

In [ ]:
# Compare isotherms
NX, NY = 2, 3
figure, axis = plt.subplots(NX, NY)
k = 0

for i in range(NX):
    for j in range(NY):
        net_isotherm = calculate_isotherm_by_distribution(gen, pore_widths, fit_exp_list[k].T[0])
        classic_isotherm = calculate_isotherm_by_distribution(gen, pore_widths, fit_classic_list[k].x)
        quantachrome_data = np.genfromtxt(f"data/real/quantachrome/silica/fitting/{exp_file_list[k]}.csv",
                                          delimiter=",")[1:].T
        quantachrome_pore_size = \
            np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[
                0] / 10 * 2
        quantachrome_dV = \
            np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[3]
        quantachrome_data2 = calculate_isotherm_by_distribution(gen, pore_widths,
                                                                np.interp(pore_widths, quantachrome_pore_size,
                                                                          quantachrome_dV))

        y_scale_factor_net = max(net_isotherm) / max(n_s_exp_raw_list[k])
        y_scale_factor_classic = max(classic_isotherm) / max(n_s_exp_raw_list[k])
        y_scale_factor_quantachrome = max(quantachrome_data[1]) / max(n_s_exp_raw_list[k])
        y_scale_factor_quantachrome2 = max(quantachrome_data2) / (
            n_s_exp_raw_list[k][-1])  #max(quantachrome_data[1]) / max(n_s_exp_raw_list[k])
        print(1 / y_scale_factor_quantachrome2)

        axis[i, j].plot(p_exp_list[k], n_s_exp_raw_list[k], label="real")
        axis[i, j].plot(gen.pressures_s, classic_isotherm / y_scale_factor_classic, marker=".", label=f"classic")
        axis[i, j].plot(gen.pressures_s, net_isotherm / y_scale_factor_net, label="net")
        # axis[i, j].plot(quantachrome_data[0], quantachrome_data[1]/y_scale_factor_quantachrome, label="quantachrome")
        # axis[i, j].plot(gen.pressures_s, quantachrome_data2/y_scale_factor_quantachrome2, label="quantachrome_from_kernal")
        axis[i, j].set_title(f"{exp_file_list[k]}")
        axis[i, j].legend(loc="lower right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
#QUANTACHROME
data = np.genfromtxt("data/real/quantachrome/silica/distribution/MIL-101.csv", delimiter=",")[1:].T[0]